## Import

In [ ]:
import os

import numpy as np

import nengo
from nengo.params import Default, NumberParam
from nengo.utils.matplotlib import rasterplot
from nengo.dists import Uniform
from nengo.solvers import LstsqL2, NoSolver
from nengo.neurons import *
from nengo.builder.neurons import *

from nengolib.signal import s, nrmse, LinearSystem
from nengolib.synapses import Lowpass

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='poster', style='whitegrid')
%matplotlib inline

## WilsonNeuron

In [ ]:
class WilsonNeuron(NeuronType):
    """
    Wilson Neuron Model.
    
    Description
    -----------
    
    TODO
    
    Paramters
    ---------
    
    time constants in seconds
    voltages in mV
    
    References
    ----------
    [1] Wilson, Hugh R. "Simplified dynamics of human and mammalian neocortical neurons."
        Journal of theoretical biology 200.4 (1999): 375-388.

    
    """
    
    probeable = ('spikes', 'voltage', 'recovery', 'conductance', 'AP')
    threshold = NumberParam('threshold')
    tau_V = NumberParam('tau_V')
    tau_R = NumberParam('tau_R')
    tau_H = NumberParam('tau_H')
    
    def __init__(self, threshold=-0.20, tau_V=0.00097, tau_R=0.0056, tau_H=0.0990):
        super(WilsonNeuron, self).__init__()
        self.threshold = threshold
        self.tau_V = tau_V
        self.tau_R = tau_R
        self.tau_H = tau_H
        
    @property
    def _argreprs(self):
        args = []
        def add(attr, default):
            if getattr(self, attr) != default:
                args.append("%s=%s" %(attr, getattr(self, attr)))
        add("threshold", -0.20)
        add("tau_V", 0.00097)
        add("tau_R", 0.0056)
        add("tau_H", 0.0990)
        return args
        
    def rates(self, x, gain, bias):
        return x  # todo
    
    def gain_bias(self, max_rates, intercepts):
        return np.ones(len(max_rates)), np.ones(len(max_rates))  # todo
    
    def step_math(self, dt, J, spiked, voltage, recovery, conductance, AP):

        dV = -(17.81 + 47.58*voltage + 33.80*np.square(voltage))*(voltage - 0.48) - 26*recovery*(voltage + 0.95) - 13*conductance*(voltage + 0.95) + J
        dR = -recovery + 1.29*voltage + 0.79 + 3.30*np.square(voltage + 0.38)
        dH = -conductance + 11*(voltage + 0.754)*(voltage + 0.69)
        
        voltage[:] = (voltage + dV * dt/self.tau_V).clip(-0.9, 0.3)
        recovery[:] = (recovery + dR * dt/self.tau_R)  # .clip(0.18, 0.42)
        conductance[:] = (conductance + dH * dt/self.tau_H)  # .clip(0, 0.23)
        spiked[:] = (voltage > self.threshold) & (~AP)
        spiked /= dt
        AP[:] = voltage > self.threshold
        
#         voltage[:] += dV * dt/self.tau_V
#         recovery[:] += dR * dt/self.tau_R
#         conductance[:] += dH * dt/self.tau_H
#         spiked[:] = (voltage > self.threshold) & (~AP)
#         spiked /= dt
#         AP[:] = voltage > self.threshold



@Builder.register(WilsonNeuron)
def build_wilsonneuron(model, neuron_type, neurons):
    model.sig[neurons]['voltage'] = Signal(
        -0.754*np.ones(neurons.size_in), name="%s.voltage" % neurons)
    model.sig[neurons]['recovery'] = Signal(
        0.279*np.ones(neurons.size_in), name="%s.recovery" % neurons)
    model.sig[neurons]['conductance'] = Signal(
        0.0*np.ones(neurons.size_in), name="%s.conductance" % neurons)
    model.sig[neurons]['AP'] = Signal(
        np.zeros(neurons.size_in, dtype=bool), name="%s.AP" % neurons)
    model.add_op(SimNeurons(
        neurons=neuron_type,
        J=model.sig[neurons]['in'],
        output=model.sig[neurons]['out'],
        states=[model.sig[neurons]['voltage'],
            model.sig[neurons]['recovery'],
            model.sig[neurons]['conductance'],
            model.sig[neurons]['AP']]))

## Utilities

In [ ]:
def norms(signal, freq, amp, ss, tau, t, dt=0.001, plot=False):
    lpf=nengo.Lowpass(tau)
    with nengo.Network() as model:
        if signal == 'cos':
            stim = nengo.Node(output=lambda t: amp*np.cos(freq*t))
        elif signal == 'white_noise':
            stim = nengo.Node(nengo.processes.WhiteSignal(period=t/2, high=freq, rms=amp, seed=ss))
        p_stim = nengo.Probe(stim, synapse=None)
        p_integral = nengo.Probe(stim, synapse=1/s)
    with nengo.Simulator(model, progress_bar=False, dt=dt) as sim:
        sim.run(t, progress_bar=False)
    stimulus = sim.data[p_stim]
    target = sim.data[p_integral]
    target_f = lpf.filt(sim.data[p_integral], dt=dt)
    norm_s = np.max(np.abs(stimulus))
    norm = np.max(np.abs(target))
    norm_f = np.max(np.abs(target_f))
    if plot:
        plt.plot(sim.trange(), stimulus, label='stim', alpha=0.5)
        plt.plot(sim.trange(), target, label='integral', alpha=0.5)
        plt.plot(sim.trange(), lpf.filt(sim.data[p_integral]/norm_f, dt=dt), label='target', alpha=0.5)
        plt.legend()
        plt.show()
    return norm, norm_s, norm_f

In [ ]:
def hyperopt_objective(hyps):
    num = 1  # constant term?
    den = 1  # constant term?
    for n in range(hyps['n_num']): # terms in transfer function's numerator
        num += s**(n+1) * hyps['%s_num'%n]
    for d in range(hyps['n_den']): # terms in transfer function's denominator
        den += s**(d+1) * hyps['%s_den'%d]  
    h = num / den
    h /= h.dcgain

    spikes = np.load(hyps['savefile'])['spikes']
    target = np.load(hyps['savefile'])['target']
    decoders = np.load(hyps['savefile'])['decoders']
    A_stack = np.zeros((len(spikes[0])*(hyps['r']+1), hyps['n_neurons']))
    for n in range(hyps['r']+1):
        A_stack[n*len(spikes[0]):(n+1)*len(spikes[0])] = h.filt(
            spikes[n], dt=hyps['dt'])[::int(hyps['dt_sample']/hyps['dt'])]
    xhat = np.dot(A_stack, decoders)
    if np.sum(target) != 0:
        loss = nrmse(xhat, target=target)
    else:
        loss = rmse(xhat, target)

    return {'loss': loss, 'h': h, 'status': STATUS_OK }

In [ ]:
def get_syn_e_init(n_neurons, n_pre, max_rates, seeds, dim=1, neuron_type=nengo.LIF()):
    syn_e_init = np.zeros((n_neurons, n_pre))
    with nengo.Network(seed=seeds['ns']) as pre_model:
        lif = nengo.Ensemble(n_neurons, dim, max_rates=max_rates, neuron_type=neuron_type, seed=seeds['es'])
    sim = nengo.Simulator(pre_model, seed=seeds['ss'])
    e_target = sim.data[lif].encoders
    for n in range(n_neurons):
        # each synaptic encoder on neuron "n" is initialized identically
        syn_e_init[n] = e_target[n] * np.ones((n_pre, ))
    return syn_e_init

In [ ]:
def bin_activities_values_1d(
    xhat_pre,
    act_bio,
    x_min=-1,
    x_max=1,
    n_neurons=10,
    n_eval_points=20):

    def find_nearest(array,value):
        idx = (np.abs(array-value)).argmin()
        return idx

    x_bins = np.linspace(x_min, x_max, num=n_eval_points)
    hz_means = np.empty((n_neurons, n_eval_points))
    hz_stds = np.empty((n_neurons, n_eval_points))
    for i in range(n_neurons):
        bin_act = [[] for _ in range(x_bins.shape[0])]
        for t in range(act_bio.shape[0]):
            idx = find_nearest(x_bins, xhat_pre[t])
            bin_act[idx].append(act_bio[t, i])
        for x in range(len(bin_act)):
            hz_means[i, x] = np.average(bin_act[x])
            hz_stds[i, x] = np.std(bin_act[x])

    return x_bins, hz_means, hz_stds

## Synaptic Weight Node

In [ ]:
class SynWNode(nengo.Node):
    def __init__(self, n_neurons, w):
        self.n_neurons = n_neurons
        self.w = w
        self.out = np.zeros((self.n_neurons, ))
        super(SynWNode, self).__init__(
            self.update, 
            size_in=self.n_neurons,  # x=filtered spikes 
            size_out=self.n_neurons)  # output=weighted value

    def update(self, t, x):
        self.out = np.dot(self.w, x).ravel()
        return self.out

## Decoder and Filter Learning

In [ ]:
def train(
    d,
    d_JL,
    e,
    w,
    h,
    r,
    stim_data,
    target_data,
    spk_list_data,
    spk_unsupv_data,
    lif_spk_data,
    lif_data,
    n_neurons=100,
    reg=0.1,
    taus={'network': 0.1, 'readout': 0.1},
    dt=0.001,
    dt_sample=0.001,
    n_num=1,
    n_den=3,
    num_min=2, 
    num_max=3, 
    den_min=1, 
    den_max=2, 
    max_evals=100,
    update=None,
    plot=False,
    **kwargs):
    
    ''' Collect spikes, activities and lif estimates '''
    lpf = nengo.Lowpass(taus['readout'])
    stim = lpf.filt(stim_data, dt=dt)[::int(dt_sample/dt)]
    target = lpf.filt(target_data, dt=dt)[::int(dt_sample/dt)]
    times = dt_sample * np.arange(0, len(target), 1)
    A = np.zeros((r+1, len(times), n_neurons))  # (unrollings, times, neurons)
    all_spikes = np.array([spk_list_data[n] for n in range(r+1)])
    A_stack = np.zeros((len(times)*(r+1), n_neurons))  # stack all unrollings into 1d
    A_stack_avg = np.zeros((len(times), n_neurons))  # average over unrollings
    A_target = np.zeros_like(A_stack)  # repeated copies of A_target to match A_stack shape
    A_unsupv = h.filt(spk_unsupv_data, dt=dt)[::int(dt_sample/dt)]  # unsupervised activities
    Y = np.zeros((r+1, len(times), 1))
    Y_stack = np.zeros((len(times)*(r+1), 1))
    for roll in range(r+1):
        A[roll] = h.filt(spk_list_data[roll], dt=dt)[::int(dt_sample/dt)]
        Y[roll] = np.array(target)
        A_stack[roll*len(times):(roll+1)*len(times)] = A[roll]
        Y_stack[roll*len(times):(roll+1)*len(times)] = Y[roll]
        A_target[roll*len(times):(roll+1)*len(times)] = lpf.filt(lif_spk_data, dt=dt)[::int(dt_sample/dt)]
    for n in range(n_neurons):
        A_stack_avg[:,n] = np.average(A[:,:,n], axis=0)
    xhat_lif = lif_data[::int(dt_sample/dt)]
    nrmse_lif = nrmse(xhat_lif, target=target)
    xhats = np.zeros_like(Y)
    nrmses = np.zeros((r+1))
    for roll in range(r+1):
        xhats[roll] = np.dot(A[roll], d)
        nrmses[roll] = nrmse(xhats[roll], target=target)
    xhat_unsupv = np.dot(A_unsupv, d)
    nrmse_unsupv = nrmse(xhat_unsupv, target=target)
    # observed tuning curves
    A_target_0 = lpf.filt(lif_spk_data, dt=dt)[::int(dt_sample/dt)]
    x_bins_lif, hz_means_lif, hz_stds_lif = bin_activities_values_1d(
        target, A_target_0, n_neurons=n_neurons)
    x_bins_unsupv, hz_means_unsupv, hz_stds_unsupv = bin_activities_values_1d(
        target, A_unsupv, n_neurons=n_neurons)

    ''' update encoders, decoders, and/or filters '''
    w_new = np.zeros_like(w)
    w_JL_new = np.zeros_like(w)
    rng = np.random.RandomState(seed=kwargs['seeds']['ls'])
    d_new = d
    e_new = e
    h_new = h

    if update == "e":  # only updates synaptic encoders for recurrent connections
        e_new = np.array(e)
        for n in range(n_neurons):
            # TODO: this error should be A[n] - A_target_0 for each unroll, squared then averaged
#             a_error = np.average(np.square(A_stack_avg[:,n] - A_target_0[:,n]))
            a_error = np.average(np.square(-A_stack_avg[:,n] + A_target_0[:,n]))
#             times = np.arange(0, A_target_0.shape[0])
#             plt.plot(times, A_target_0[:,n], label='target')
#             plt.plot(times, A_stack_avg[:,n], label='average over r, neuron %s, error=%.3f' %(n, a_error))
#             plt.legend()
#             plt.show()
#             a_error = np.square(np.average(A_stack[:, n] - A_target[:, n]))
#             a_error = nrmse(A_stack[:, n], target=A_target[:, n])
            for pre in range(n_neurons):
                delta = rng.uniform(0, 2 * kwargs['eta'] * a_error)
                if a_error > 0 and d[pre] > 0:  # overactive, positive dec => reduce enc reduce weight
                    e_new[n, pre] += -delta
                if a_error > 0 and d[pre] < 0:  # overactive, negative dec => increase enc reduce weight
                    e_new[n, pre] += +delta
                if a_error < 0 and d[pre] > 0:  # underactive, positive dec => increase enc increase weight
                    e_new[n, pre] += +delta
                if a_error < 0 and d[pre] < 0:  # underactive, negative dec => reduce enc increase weight
                    e_new[n, pre] += -delta
        print("a_error_total=%.3f" %np.average(np.square(A_stack_avg - A_target_0)))

    if update == "d":
        d_new = LstsqL2(reg=reg)(A_stack, Y_stack)[0]
        
    if update == "h":
        savefile = "/home/pduggins/nengo_bioneurons/nengo_bioneurons/tests/data/trinity/stacked_spikes_targets.npz"
        np.savez(savefile, spikes=all_spikes, target=Y_stack, decoders=d)
        hyps = dict(
            savefile=savefile,
            n_num=n_num,
            n_den=n_den,
            r=r,
            n_neurons=n_neurons,
            dt=dt,
            dt_sample=dt_sample)
        for num in range(n_num):  
            hyps['%s_num'%num] = hp.uniform('%s_num'%num, num_min, num_max)
        for den in range(n_den):
            hyps['%s_den'%den] = hp.uniform('%s_den'%den, den_min, den_max)
        trials = Trials()
        best = fmin(hyperopt_objective,
            rstate=np.random.RandomState(seed=kwargs['seeds']['ss']+r),
            space=hyps,
            algo=tpe.suggest,
            max_evals=max_evals,
            trials=trials)
        best_idx = np.array(trials.losses()).argmin()
        h_new = trials.trials[best_idx]['result']['h']

    for n in range(n_neurons):
        for pre in range(n_neurons):
            w_new[n, pre] = np.dot(d_new[pre], e_new[n, pre])
            w_JL_new[n, pre] = np.dot(d_JL[pre], e_new[n, pre])

    ''' Plot 'em, mash 'em, stick 'em in a stew: filters, tuning curve match, decoded values'''
    if plot:
        impulse_time = np.arange(0, 1, 0.001)
        fig, ax = plt.subplots(1, 1, figsize=(8, 4))
        ax.plot(impulse_time, Lowpass(taus['readout']).impulse(len(impulse_time)), label='lowpass')
        ax.plot(impulse_time, h_new.impulse(len(impulse_time)), label='optimized')
        ax.legend()
        plt.tight_layout()
        plt.show()
        
        cmap = sns.color_palette('hls', 10)
        a_error = np.average(np.square(A_unsupv - A_target_0))
#         a_error = nrmse(A_stack, target=A_target)
        fig, (ax4, ax5) = plt.subplots(1, 2, figsize=(8, 8), sharey=True)
        for n in range(10):  # n_neurons
            ax4.plot(x_bins_lif, hz_means_lif[n], c=cmap[n])
            ax4.fill_between(x_bins_lif,
                hz_means_lif[n]+hz_stds_lif[n],
                hz_means_lif[n]-hz_stds_lif[n],
                alpha=0.5, facecolor=cmap[n])
            ax5.plot(x_bins_unsupv, hz_means_unsupv[n], c=cmap[n])
            ax5.fill_between(x_bins_unsupv,
                hz_means_unsupv[n]+hz_stds_unsupv[n],
                hz_means_unsupv[n]-hz_stds_unsupv[n],
                alpha=0.5, facecolor=cmap[n])
        ax4.set(xlim=((-1,1)), xlabel='$\mathbf{x}$', ylabel='activity (Hz)', title='target')
        ax5.set(xlim=((-1,1)), xlabel='$\mathbf{x}$', title='unsupv, rmse=%.3f' %a_error)
        plt.tight_layout()

        cmap = sns.color_palette('hls', r)
        fig, ax = plt.subplots(1, 1, figsize=(8, 4))
        ax.plot(times, target, label='target', linestyle='--')
        ax.plot(times, xhat_lif, alpha=0.5, label='lif, nrmse=%.3f' %nrmse_lif)
        for n in range(r+1):
            ax.plot(times, xhats[n], alpha=0.5, label='unroll %s, nrmse=%.3f' %(n, nrmses[n]))
        ax.plot(times, xhat_unsupv, label='unsupervised, nrmse=%.3f' %nrmse_unsupv)
#         if d_JL.shape[1] > 0:
#             ax.plot(times, np.dot(A_unsupv, d_JL), label='JL dim')
        ax.set(xlabel='time', ylabel='$\mathbf{x}$')
        ax.legend(loc='lower left')
        plt.tight_layout()
        plt.show()

#         fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 8), sharex=True)
#         rasterplot(sim.trange(), spikes_unsupv, ax=ax1)
#         ax2.plot(times, lpf.filt(sim.data[p_spike_list[-1]], dt=dt_sample))
#         ax2.plot(sim.trange(), lpf.filt(spikes_unsupv, dt=dt))
#         ax1.set(xlabel='time', ylabel='neuron')
#         ax2.set(xlabel='time', ylabel='firing rate (Hz)')
#         plt.tight_layout()

#         fig, ax1 = plt.subplots(1, 1, figsize=(8, 4))
#         for n in range(r+1):
#             sns.distplot(np.ravel(A[n]), ax=ax1, label='%s unrolling' %n)
#         sns.distplot(np.ravel(A_unsupv), ax=ax1, label='unsupervised', hist=True)
#         ax1.set(xlim=((1, 400)), ylim=((0, 0.03)), xlabel='firing rate (Hz)', ylabel='frequency')
#         plt.legend()
#         plt.tight_layout()

#         times = np.arange(0, 1, 0.001)
#         xs = [t['tid'] for t in trials.trials]
#         ys = [t['result']['loss'] for t in trials.trials]
#         fig, (ax, ax2) = plt.subplots(2, 1, figsize=((8, 8)))
#         ax.scatter(xs, ys)
#         ax.set(xlabel='hyperopt iteration', ylabel='nrmse')
#         ax2.plot(times, Lowpass(tau).impulse(len(times)), label='lowpass')
#         ax2.plot(times, best_h.impulse(len(times)), label='optimized')
#         plt.tight_layout()
#         plt.show()

    return d_new, e_new, h_new, w_new, w_JL_new, nrmse_unsupv

# Network

In [ ]:
def simulate(  
    w,
    w_JL,
    h,
    r,
    t=1,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    signal='cos',
    freq=1,
    amp=1,
    reg=0.01,
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    neuron_type=nengo.LIF(),
    max_rates=Default,
    gain=Default,
    bias=Default,
    **kwargs):
    
    # transform input signal u so that the integral x is normalized to np.max(x)==1
    norm, norm_s, norm_f = norms(signal, freq, amp, seeds['ss'], taus['network'], t, dt=dt, plot=False)
    
    # Simulate the full network, skipping simulation of supv or bio when possible.
    with nengo.Network(seed=seeds['ns']) as model:
        # Nodes
        model.T = t
        def flip(t, x):
            if t<model.T/2: return x
            elif t>=model.T/2: return -1.0*x
        if signal == 'cos':
            u_raw = nengo.Node(output=lambda t: amp*np.cos(freq*t))
        elif signal == 'white_noise':
            u_raw = nengo.Node(nengo.processes.WhiteSignal(period=model.T/2, high=freq, rms=amp, seed=seeds['ss']))
        u = nengo.Node(output=flip, size_in=1)
        node_list = []
        node_JL_list = []
        for n in range(r):
            node_list.append(SynWNode(n_neurons, w))
            node_JL_list.append(SynWNode(n_neurons, w_JL))
        unsupv_node = SynWNode(n_neurons, w)
        unsupv_node_JL = SynWNode(n_neurons, w_JL)
                                              
        # Ensembles
        pre_u = nengo.Ensemble(n_pre, 1, max_rates=max_rates, seed=seeds['es'], radius=norm_s)
        pre_x = nengo.Ensemble(n_pre, 1, max_rates=max_rates, seed=seeds['es'], radius=norm)
        ens_list = []
        for n in range(r+1):
            ens_list.append(
                nengo.Ensemble(n_neurons, 1,
                    gain=gain, bias=bias, max_rates=max_rates,
                    neuron_type=neuron_type, seed=seeds['es'], label='%s'%n))
        unsupv = nengo.Ensemble(n_neurons, 1,
                gain=gain, bias=bias, max_rates=max_rates,
                neuron_type=neuron_type, seed=seeds['es'], label='unsupv')
        lif = nengo.Ensemble(n_neurons, 1, max_rates=max_rates, neuron_type=nengo.LIF(), seed=seeds['es'])
        tar = nengo.Ensemble(1, 1, neuron_type=nengo.Direct())

        # normal connections
        nengo.Connection(u_raw, u, synapse=None, seed=seeds['cs'])
        nengo.Connection(u, pre_u, synapse=None, seed=seeds['cs'])
        nengo.Connection(u, pre_x, synapse=1/s, seed=seeds['cs'])
        nengo.Connection(u, tar, synapse=1/s, transform=1.0/norm_f)
        nengo.Connection(pre_u, lif, synapse=taus['network'], transform=taus['network']/norm_f)
#         nengo.Connection(pre_x, lif, synapse=taus['network'], transform=1.0/norm_f)  # proxy for accuracy
        nengo.Connection(lif, lif, synapse=taus['network'])
        
        # supervised connections (train)
        u_ens_list = []
        for n in range(r+1):
            u_ens_list.append(nengo.Connection(pre_u, ens_list[n][0],
                synapse=taus['network'],
                transform=taus['network']/norm_f,
                seed=seeds['cs']))
        ens_node_list = []
        node_ens_list = []
        ens_node_JL_list = []
        node_JL_ens_list = []
        for n in range(r):
            ens_node_list.append(nengo.Connection(ens_list[n].neurons, node_list[n], synapse=h))
            node_ens_list.append(nengo.Connection(node_list[n], ens_list[n+1].neurons, synapse=None))
            ens_node_JL_list.append(nengo.Connection(ens_list[n].neurons, node_JL_list[n], synapse=h))
            node_JL_ens_list.append(nengo.Connection(node_JL_list[n], ens_list[n+1].neurons, synapse=None))

        nengo.Connection(pre_x, ens_list[0][0],
            synapse=taus['network'],
            transform=1.0/norm_f,
            seed=seeds['cs'])
        
        # unsupervised connections (test)
        nengo.Connection(pre_u, unsupv[0],
            synapse=taus['network'],
            transform=taus['network']/norm_f)
        nengo.Connection(unsupv.neurons, unsupv_node, synapse=h)
        nengo.Connection(unsupv_node, unsupv.neurons, synapse=None)
        nengo.Connection(unsupv.neurons, unsupv_node_JL, synapse=h)
        nengo.Connection(unsupv_node_JL, unsupv.neurons, synapse=None)


        # probes (down sample to dt_sample in post-processing for spiking ensembles)
        p_stim = nengo.Probe(u, synapse=None, sample_every=dt)
        p_target = nengo.Probe(tar, synapse=None, sample_every=dt)
        p_spk_list = []
        for n in range(r+1):
            p_spk_list.append(nengo.Probe(ens_list[n].neurons, synapse=None, sample_every=dt))
        p_spk_unsupv = nengo.Probe(unsupv.neurons, synapse=None, sample_every=dt)
        p_spk_lif = nengo.Probe(lif.neurons, sample_every=dt)
        p_lif = nengo.Probe(lif, synapse=taus['readout'], sample_every=dt)
        

    # Run the simulation
    with nengo.Simulator(model, seed=seeds['ss'], dt=dt, progress_bar=False) as sim:
        sim.run(t, progress_bar=True)
        
    spk_list = [sim.data[p_spk_list[n]] for n in range(r+1)]
        
    return sim.data[p_stim], sim.data[p_target], spk_list, sim.data[p_spk_unsupv], sim.data[p_spk_lif], sim.data[p_lif]

## Control Structure

In [ ]:
def run(d_updates=0, e_updates=0, h_updates=0, r_updates=0, plot=False, signal='cos', **kwargs):
    rng = np.random.RandomState(seed=kwargs['seeds']['ls'])

    d = np.zeros((kwargs['n_neurons'], 1))
    d_JL = rng.randn(kwargs['n_neurons'], 1) * kwargs['jl_mag']
    e = get_syn_e_init(kwargs['n_neurons'], kwargs['n_neurons'], kwargs['max_rates'], seeds=kwargs['seeds'])
    h = Lowpass(kwargs['taus']['readout'])
    w = np.zeros((kwargs['n_neurons'], kwargs['n_neurons']))
    w_JL = np.zeros((kwargs['n_neurons'], kwargs['n_neurons']))
    for n in range(kwargs['n_neurons']):
        for pre in range(kwargs['n_neurons']):
            w[n, pre] = np.dot(d[pre], e[n, pre])
            w_JL[n, pre] = np.dot(d_JL[pre], e[n, pre])
            
    updates = []
    for i in range(d_updates): updates.append('d')
    for i in range(e_updates): updates.append('e')
    for i in range(h_updates): updates.append('h')
#     rng.shuffle(updates)
    
    for r in range(r_updates):
        print("unrolling %s" %r)
        for update in updates:
            print("updating %s" %update)
            sd, td, sld, sud, lsd, ld = simulate(w, w_JL, h, r, signal=signal, **kwargs)
            d, e, h, w, w_JL, nrmse_unsupv = train(d, d_JL, e, w, h, r, sd, td, sld, sud, lsd, ld,
                update=update, plot=plot, **kwargs)
        
    # test
    kwargs['seeds']['ss'] = kwargs['seeds']['ss'] + 1
    sd, td, sld, sud, lsd, ld = simulate(w, w_JL, h, r, signal='white_noise', **kwargs)
    d, e, h, w, w_JL, nrmse_unsupv = train(d, d_JL, e, w, h, 0, sd, td, sld, sud, lsd, ld, plot=True, **kwargs)    
    
    return d, e, h

# Simulations

## LIF

### baseline

In [ ]:
kwargs = dict(
    t=4*np.pi,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.LIF(),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    reg=1e-2,
    eta=1e-4,
    jl_mag=0e-4,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=100)

d, e, h = run(d_updates=1, e_updates=0, h_updates=0, r_updates=3, plot=True, signal='cos', **kwargs)

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.LIF(),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    reg=1e-2,
    eta=1e-4,
    jl_mag=0e-4,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=100)

d, e, h = run(d_updates=1, e_updates=0, h_updates=0, r_updates=3, plot=True, signal='white_noise', **kwargs)

### JL dimensions

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.LIF(),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    reg=1e-2,
    eta=1e-4,
    jl_mag=1e-6,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=100)

d, e, h = run(d_updates=1, e_updates=0, h_updates=0, r_updates=3, plot=True, signal='white_noise', **kwargs)

### Filter

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.LIF(),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    reg=1e-2,
    eta=1e-4,
    jl_mag=0e-6,
    n_num=0, 
    n_den=1,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=100)

d, e, h = run(d_updates=1, e_updates=0, h_updates=1, r_updates=3, plot=True, signal='white_noise', **kwargs)

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.LIF(),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    reg=1e-2,
    eta=1e-4,
    jl_mag=0e-6,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d, e, h = run(d_updates=1, e_updates=0, h_updates=1, r_updates=3, plot=True, signal='white_noise', **kwargs)

### Encoder

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.LIF(),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    reg=1e-2,
    eta=1e-4,
    jl_mag=0e-6,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=100)

d, e, h = run(d_updates=1, e_updates=4, h_updates=0, r_updates=3, plot=True, signal='white_noise', **kwargs)

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.LIF(),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    reg=1e-2,
    eta=1e-6,
    jl_mag=0e-6,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=100)

d, e, h = run(d_updates=1, e_updates=5, h_updates=0, r_updates=2, plot=True, signal='white_noise', **kwargs)

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.LIF(),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    reg=1e-2,
    eta=1e-4,
    jl_mag=0e-6,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=100)

d, e, h = run(d_updates=1, e_updates=10, h_updates=0, r_updates=2, plot=False, signal='white_noise', **kwargs)

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.LIF(),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    reg=1e-2,
    eta=1e-4,
    jl_mag=0e-6,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=100)

d, e, h = run(d_updates=1, e_updates=5, h_updates=0, r_updates=2, plot=False, signal='white_noise', **kwargs)

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.LIF(),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    reg=1e-2,
    eta=1e-3,
    jl_mag=0e-6,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=100)

d, e, h = run(d_updates=1, e_updates=4, h_updates=0, r_updates=3, plot=True, signal='white_noise', **kwargs)

### JL, Filter, and Encoder

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.LIF(),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    reg=1e-2,
    eta=1e-4,
    jl_mag=1e-6,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d, e, h = run(d_updates=2, e_updates=1, h_updates=1, r_updates=3, plot=True, signal='white_noise', **kwargs)

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.LIF(),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=1,
    jl_sigma=1e-4,
    reg=0.01,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=6, signal='white_noise', optimize=True, plot=True, **kwargs)

## ALIF

### baseline

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=0,
    jl_sigma=1e-4,
    reg=0.01,
    n_num=0, 
    n_den=1,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=100)

d_best, h_best = unroll_best(r=6, signal='white_noise', optimize=False, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=8*np.pi,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=0,
    jl_sigma=2e-4,
    reg=0.01,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=10, signal='cos', optimize=False, plot=True, **kwargs)

### regularization

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=0,
    jl_sigma=1e-4,
    reg=0.1,
    n_num=0, 
    n_den=1,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=100)

d_best, h_best = unroll_best(r=6, signal='white_noise', optimize=False, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=0,
    jl_sigma=1e-4,
    reg=0.001,
    n_num=0, 
    n_den=1,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=100)

d_best, h_best = unroll_best(r=6, signal='white_noise', optimize=False, plot=True, **kwargs)

### JL dims

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=1,
    jl_sigma=1e-4,
    reg=0.01,
    n_num=0, 
    n_den=1,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=100)

d_best, h_best = unroll_best(r=6, signal='white_noise', optimize=False, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=3,
    jl_sigma=1e-4,
    reg=0.01,
    n_num=0, 
    n_den=1,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=100)

d_best, h_best = unroll_best(r=6, signal='white_noise', optimize=False, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=1,
    jl_sigma=3e-4,
    reg=0.01,
    n_num=0, 
    n_den=1,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=100)

d_best, h_best = unroll_best(r=6, signal='white_noise', optimize=False, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=3,
    jl_sigma=3e-4,
    reg=0.01,
    n_num=0, 
    n_den=1,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=100)

d_best, h_best = unroll_best(r=6, signal='white_noise', optimize=False, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=3,
    jl_sigma=3e-4,
    reg=0.1,
    n_num=0, 
    n_den=1,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=100)

d_best, h_best = unroll_best(r=6, signal='white_noise', optimize=False, plot=True, **kwargs)

### optimized filter

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=0,
    jl_sigma=1e-4,
    reg=0.01,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=300)

d_best, h_best = unroll_best(r=6, signal='white_noise', optimize=True, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=0,
    jl_sigma=1e-4,
    reg=0.1,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = unroll_best(r=6, signal='white_noise', optimize=True, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=0,
    jl_sigma=1e-4,
    reg=0.01,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=6, signal='white_noise', optimize=True, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=0,
    jl_sigma=1e-4,
    reg=0.1,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=6, signal='white_noise', optimize=True, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=0,
    jl_sigma=1e-4,
    reg=0.001,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=6, signal='white_noise', optimize=True, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=8*np.pi,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=0,
    jl_sigma=1e-4,
    reg=0.01,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=6, signal='cos', optimize=True, plot=True, **kwargs)

### filter and JL dims

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=1,
    jl_sigma=1e-4,
    reg=0.001,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=8, signal='white_noise', optimize=True, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=1,
    jl_sigma=1e-4,
    reg=0.01,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=8, signal='white_noise', optimize=True, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=1,
    jl_sigma=1e-4,
    reg=0.1,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=8, signal='white_noise', optimize=True, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=1,
    jl_sigma=3e-4,
    reg=0.001,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=8, signal='white_noise', optimize=True, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=1,
    jl_sigma=2e-4,
    reg=0.003,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=6, signal='white_noise', optimize=True, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=2,
    jl_sigma=2e-4,
    reg=0.003,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=6, signal='white_noise', optimize=True, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=3,
    jl_sigma=3e-4,
    reg=0.003,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=8, signal='white_noise', optimize=True, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=30,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=3,
    jl_sigma=2e-4,
    reg=0.005,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=10, signal='white_noise', optimize=True, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=30,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=3,
    jl_sigma=2e-4,
    reg=0.001,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=8, signal='white_noise', optimize=True, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=30,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=1,
    jl_sigma=4e-4,
    reg=0.001,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=8, signal='white_noise', optimize=True, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=1,
    jl_sigma=2e-4,
    reg=0.001,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=8, signal='white_noise', optimize=True, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=2*np.pi,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=1,
    jl_sigma=2e-4,
    reg=0.001,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=8, signal='cos', optimize=True, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=2*np.pi,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=1,
    jl_sigma=2e-4,
    reg=0.01,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=8, signal='cos', optimize=True, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=10,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=2*np.pi,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=2,
    jl_sigma=2e-4,
    reg=0.01,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=8, signal='cos', optimize=True, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=8*np.pi,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=1,
    jl_sigma=2e-4,
    reg=0.01,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=6, signal='cos', optimize=True, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=8*np.pi,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=2,
    jl_sigma=2e-4,
    reg=0.01,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=6, signal='cos', optimize=True, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=8*np.pi,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=1,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=3,
    jl_sigma=2e-4,
    reg=0.01,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=6, signal='cos', optimize=True, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=8,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=2*np.pi,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=1,
    jl_sigma=2e-4,
    reg=0.01,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=6, signal='cos', optimize=True, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=8,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=2*np.pi,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=2,
    jl_sigma=2e-4,
    reg=0.01,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=6, signal='cos', optimize=True, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=8,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=2*np.pi,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=3,
    jl_sigma=2e-4,
    reg=0.01,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=6, signal='cos', optimize=True, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=16,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=np.pi,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=3,
    jl_sigma=3e-4,
    reg=0.01,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=6, signal='cos', optimize=True, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=8,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=0.5*np.pi,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=3,
    jl_sigma=1e-4,
    reg=0.001,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=6, signal='cos', optimize=True, plot=True, **kwargs)

In [ ]:
kwargs = dict(
    t=16,
    dt=0.001,
    dt_sample=0.001,
    n_neurons=100,
    n_pre=100,
    freq=0.5*np.pi,
    amp=1,
    gain=Default,
    bias=Default,
    max_rates=Default,
    neuron_type=nengo.AdaptiveLIF(tau_n=0.1, inc_n=0.1),
    taus={'network': 0.1, 'readout': 0.1},
    seeds={'ns': 1, 'ss':2, 'es': 3, 'cs': 4, 'ls': 5},
    jl_dim=3,
    jl_sigma=1e-4,
    reg=0.001,
    n_num=1, 
    n_den=3,
    num_min=0,
    num_max=1,
    den_min=0,
    den_max=1,
    max_evals=1000)

d_best, h_best = filter_first(r=6, signal='cos', optimize=True, plot=True, **kwargs)